In [1]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
load_dotenv()
groq_key = os.getenv("GROQ")
groq_key

'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'

In [6]:
groq_model = ChatGroq(
    model="llama3-70b-8192",
    groq_api_key=groq_key)

In [8]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma, FAISS
from langchain.embeddings import HuggingFaceEmbeddings

In [10]:
loader = TextLoader(
    "D:/training/ml_algorithms/my_pet.txt"
)
text_file = loader.load()
reccursive_text_splitter_txt = RecursiveCharacterTextSplitter(
    chunk_size=25,
    chunk_overlap=2
)
texts_txt_reccursive = reccursive_text_splitter_txt.split_text(text_file[0].page_content)
texts_txt_reccursive
texts_txt_reccursive_chunks = reccursive_text_splitter_txt.split_documents(text_file)
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en")
# Create ChromaDB vector store using FAISS
vector_db_reccursive = FAISS.from_documents(
    documents=texts_txt_reccursive_chunks,
    embedding=embedding_model  # This now uses the correct embedding class
)
retriver = vector_db_reccursive.as_retriever(search_kwargs={"k": 2})
question_txt = "What is the nature of my pet dog?"
response = retriver.invoke(question_txt)
print(response)


[Document(id='e5b97fce-d9c5-4289-9e47-d82a5c37c9ee', metadata={'source': 'D:/training/ml_algorithms/my_pet.txt'}, page_content='a friendly nature that'), Document(id='0dc0081e-7fe9-47ae-8c6c-255d76c4d662', metadata={'source': 'D:/training/ml_algorithms/my_pet.txt'}, page_content='him not just a pet but a')]


In [13]:
template = """Answer question based on on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [16]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
## TO get the page contents
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])
chain = (
    {
        "context": retriver | format_docs, 
        "question": RunnablePassthrough()
    }
    | prompt
    | groq_model
    | StrOutputParser()
)

In [18]:
response_final = chain.invoke(question_txt)
print(response_final)

Based on the context, the answer is: a friendly nature.
